### Librerías a Útilizar

---

In [35]:
import numpy as np
import random
import string

### Objeto Tablero

---

**El Objeto Tablero (`Table`) debe de tener las siguientes caracteristicas:**

* Debe generar letras aleatoriamente, desde la A, hasta la Z (Sin la inclusión de la Ñ)
* Se debe de asegurar de que haya posibilidad de combinación de los `keywords`, que son:
    - `LOK`
    - `TLAK`
    - `TAK` 

In [38]:
class Table:
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns
        # Usa dtype='U1' para string de un carácter en numpy
        self.data = np.zeros((rows, columns), dtype='U1')  
        self._valid_letters = list(string.ascii_uppercase)

    def set_letter(self):
        for i in range(self.rows):
            for j in range(self.columns):
                self.data[i][j] = random.choice(self._valid_letters)

    def mostrar(self):
        for fila in self.data:
            print(' '.join(fila))

In [45]:
tabla = Table(6, 6)
tabla.set_letter()
tabla.mostrar()

C D F M I Z
A G G W M S
B K F Z N N
A X L A N E
E M V Q C U
I A Y C W H
